In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pyvista as pv
import shapely as shp
from IPython.display import display
from ipywidgets import IntProgress

In [9]:
layers = gpd.read_file(
    "/Users/jonasflor/Desktop/test_Proseg3D_Paraview/cell-polygons-layers.geojson"
)
layers.geometry = [
    shp.force_3d(x["geometry"], z=x["layer"] * 3.5) for ind, x in layers.iterrows()
]
metadata = pd.read_csv(
    "/Users/jonasflor/Desktop/test_Proseg3D_Paraview/adata_obs_annotated.csv"
)

In [21]:
def shapely_to_pyvista(shapely_geometry):
    # Check if it's a multipolygon
    if isinstance(shapely_geometry, shp.geometry.MultiPolygon):
        coords = []
        for poly in list(shapely_geometry.geoms):
            coords_new = poly.exterior.coords
            coords.append(coords_new)
        # Return a list of meshes for the multipolygon
        return coords
    else:
        # If it's just a single polygon
        coords = shapely_geometry.exterior.coords
        return coords


layers["coords"] = [shapely_to_pyvista(x) for x in layers.geometry]

In [27]:
id = 0
points = []
cell_geometry = layers[layers.cell == id]
for _, row in cell_geometry.iterrows():
    for seq in row.coords:
        for coord in seq:
            points.append([coord[0], coord[1], coord[2]])
mesh_3d = pv.PolyData(points)
triagulation_old = mesh_3d.reconstruct_surface()
triagulation = triagulation_old.decimate_pro(0.3, preserve_topology=True)
final_mesh = triagulation.fill_holes(1e6)
final_mesh.clean(inplace=True)
final_mesh.plot()

Widget(value='<iframe src="http://localhost:59201/index.html?ui=P_0x35ad2a4d0_0&reconnect=auto" class="pyvista…

In [28]:
for ct in list(np.unique(metadata.cell_type_final)):
    blocks_uncapped = pv.MultiBlock()
    blocks_capped = pv.MultiBlock()
    ids = metadata[metadata.cell_type_final == ct].cell.unique()
    f = IntProgress(min=0, max=len(ids))  # instantiate the bar
    display(f)
    for id in ids:
        points = []
        cell_geometry = layers[layers.cell == id]
        for _, row in cell_geometry.iterrows():
            for seq in row.coords:
                for coord in seq:
                    points.append([coord[0], coord[1], coord[2]])
        mesh_3d = pv.PolyData(points)
        triagulation = mesh_3d.reconstruct_surface()
        triagulation = triagulation.decimate_pro(0.3, preserve_topology=True)
        final_mesh = triagulation.fill_holes(1e6)
        final_mesh.clean(inplace=True)
        blocks_capped[str(id)] = final_mesh
        blocks_uncapped[str(id)] = triagulation
        f.value += 1
    blocks_uncapped.save(
        f"/Users/jonasflor/Desktop/test_Proseg3D_Paraview/{ct}_shapes.vtm"
    )
    blocks_capped.save(
        f"/Users/jonasflor/Desktop/test_Proseg3D_Paraview/{ct}_shapes_capped.vtm"
    )

IntProgress(value=0, max=10622)

IntProgress(value=0, max=459)

IntProgress(value=0, max=5258)

IntProgress(value=0, max=2947)

IntProgress(value=0, max=293)

...

IntProgress(value=0, max=11105)

IntProgress(value=0, max=1263)

IntProgress(value=0, max=739)

IntProgress(value=0, max=10455)

IntProgress(value=0, max=926)

# Transcripts

In [17]:
transcripts = pd.read_csv(
    "/Users/jonasflor/Desktop/test_Proseg3D_Paraview/transcript-metadata.csv"
)
min_transcripts = transcripts.z.min()
max_transcripts = transcripts.z.max()
transcripts["z"] = (
    3.5 * 4 * (transcripts.z - min_transcripts) / (max_transcripts - min_transcripts)
)
transcripts

,transcript_id,x,y,z,observed_x,observed_y,observed_z,gene,qv,fov,assignment,probability,background,confusion
0,0,7136.75200,2356.6455,0.782579,7136.47560,2356.3298,0.0,Igf2,inf,461,86030,0.52001,1,0
1,1,7137.29300,2358.9102,3.766153,7138.10940,2360.3838,0.0,Cfp,inf,461,86030,0.86001,0,0
2,2,7134.17530,2359.4128,0.517902,7136.97270,2356.9917,0.0,Slc3a2,inf,461,4294967295,0.53000,0,0
3,3,7135.23970,2356.6104,1.509655,7135.13600,2357.4858,0.0,Slc3a2,inf,461,4294967295,0.33000,0,0
4,4,7138.23300,2358.5337,1.636675,7137.83640,2359.2458,0.0,Slc47a1,inf,461,86030,0.28001,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66676075,66676350,366.67264,2474.5645,3.902482,365.51303,2480.4158,4.0,Slc7a11,inf,1302,4294967295,1.00001,1,0
66676076,66676351,348.59390,2396.3267,2.299710,354.38904,2393.4180,4.0,Spp1,inf,1302,4294967295,1.00001,1,0
66676077,66676352,332.20673,2545.5134,4.684248,333.12042,2543.9292,4.0,Xirp1,inf,1302,4294967295,1.00001,1,0
66676078,66676353,245.13930,2495.4607,4.407417,245.56311,2495.7786,5.0,Slc7a11,inf,1302,4294967295,1.00001,1,0


In [18]:
cell_types_mouse_brain = {
    "ECs": ["Cldn5", "Cdh5", "Pecam1", "Kdr", "Flt1", "Tie1", "Tek", "Nos3"],
    "Pericytes": ["Vtn", "Kcnj8", "Rgs5", "Cd248", "Atp13a5", "Anpep"],
    "SMCs": ["Acta2", "Pdlim3", "Tagln", "Myh11", "Sncg", "Cnn1", "Pln", "Adamts1"],
    "Fibroblasts": ["Dcn", "Col1a1", "Lama1", "Fn1", "Slc7a11", "Il33", "Slc6a13"],
    "Ependymal cells": ["Ccdc153", "Tmem212", "Pifo", "Foxj1"],
    "Microglia_Macrophages": [
        "Trem2",
        "Cx3cr1",
        "Aif1",
        "Itgam",
        "Itgax",
        "C1qc",
        "Ctss",
        "C1qa",
        "Cx3cr1",
        "Ctss",
        "Cd163",
    ],
    "Oligodendrocytes": [
        "Plp1",
        "Cnp",
        "Cldn11",
        "Mbp",
        "Mag",
        "Mog",
        "Enpp2",
        "Trf",
        "Apod",
        "Plxnb3",
    ],
    "OPCs": [
        "Ptprz1",
        "Pdgfra",
        "Cspg4",
        "Olig1",
        "Cspg5",
        "Rgcc",
        "Olig2",
        "Vcan",
        "Cd9",
        "Sox10",
        "Apod",
    ],
    "Astrocytes": ["Aqp4", "Aldoc", "Apoe", "Clu", "Plpp3", "Slc1a3"],
    "Neurons": [
        "Meg3",
        "Slc17a7",
        "Ptprd",
        "Gad2",
        "Gad1",
        "Galntl6",
        "Rbfox3",
        "Map2",
        "Slc32a1",
    ],
}

In [19]:
f = IntProgress(min=0, max=len(cell_types_mouse_brain.keys()))  # instantiate the bar
display(f)
for key in cell_types_mouse_brain.keys():
    markers = cell_types_mouse_brain[key]
    marker_genes = transcripts[[tr in markers for tr in transcripts.gene]][
        ["x", "y", "z", "gene"]
    ].values
    points = pv.PolyData(marker_genes[:, :3].astype(float))
    points["classification"] = marker_genes[:, 3].astype(str)
    points.save(
        f"/Users/jonasflor/Desktop/test_Proseg3D_Paraview/{key}_marker_transcripts.vtp"
    )
    f.value += 1  # len(marker_genes)

IntProgress(value=0, max=10)